In [1]:
from common import *

cursor = connect()

# Recursive views

递归视图是定义查询引用视图名称本身的视图。递归视图可用于对数据库中存储的分层数据结构执行分层或递归查询。
```
CREATE RECURSIVE VIEW view_name(columns) 
AS
query;
```

等同于以下语句
```
CREATE VIEW view_name 
AS
  WITH RECURSIVE cte_name (columns) AS (
    SELECT ...)
  SELECT columns FROM cte_name;
```

In [2]:
sql = """
CREATE TABLE employees (
  employee_id SERIAL PRIMARY KEY, 
  full_name VARCHAR NOT NULL, 
  manager_id INT
);

INSERT INTO employees (employee_id, full_name, manager_id) 
VALUES 
  (1, 'Michael North', NULL), 
  (2, 'Megan Berry', 1), 
  (3, 'Sarah Berry', 1), 
  (4, 'Zoe Black', 1), 
  (5, 'Tim James', 1), 
  (6, 'Bella Tucker', 2), 
  (7, 'Ryan Metcalfe', 2), 
  (8, 'Max Mills', 2), 
  (9, 'Benjamin Glover', 2), 
  (10, 'Carolyn Henderson', 3), 
  (11, 'Nicola Kelly', 3), 
  (12, 'Alexandra Climo', 3), 
  (13, 'Dominic King', 3), 
  (14, 'Leonard Gray', 4), 
  (15, 'Eric Rampling', 4), 
  (16, 'Piers Paige', 7), 
  (17, 'Ryan Henderson', 7), 
  (18, 'Frank Tucker', 8), 
  (19, 'Nathan Ferguson', 8), 
  (20, 'Kevin Rampling', 8);
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost port=15432 user=postgres database=dvdrental) at 0x20f614e5850>

In [3]:
sql = """
WITH RECURSIVE reporting_line AS (
  SELECT 
    employee_id, 
    full_name AS subordinates 
  FROM 
    employees 
  WHERE 
    manager_id IS NULL 
  UNION ALL 
  SELECT 
    e.employee_id, 
    (
      rl.subordinates || ' > ' || e.full_name
    ) AS subordinates 
  FROM 
    employees e 
    INNER JOIN reporting_line rl ON e.manager_id = rl.employee_id
) 
SELECT 
  employee_id, 
  subordinates 
FROM 
  reporting_line 
ORDER BY 
  employee_id;
"""
run_sql(cursor, sql)

    employee_id                                       subordinates
0             1                                      Michael North
1             2                        Michael North > Megan Berry
2             3                        Michael North > Sarah Berry
3             4                          Michael North > Zoe Black
4             5                          Michael North > Tim James
5             6         Michael North > Megan Berry > Bella Tucker
6             7        Michael North > Megan Berry > Ryan Metcalfe
7             8            Michael North > Megan Berry > Max Mills
8             9      Michael North > Megan Berry > Benjamin Glover
9            10    Michael North > Sarah Berry > Carolyn Henderson
10           11         Michael North > Sarah Berry > Nicola Kelly
11           12      Michael North > Sarah Berry > Alexandra Climo
12           13         Michael North > Sarah Berry > Dominic King
13           14           Michael North > Zoe Black > Leonard 

In [4]:
sql = """
CREATE RECURSIVE VIEW reporting_line (employee_id, subordinates) AS 
SELECT 
  employee_id, 
  full_name AS subordinates 
FROM 
  employees 
WHERE 
  manager_id IS NULL 
UNION ALL 
SELECT 
  e.employee_id, 
  (
    rl.subordinates || ' > ' || e.full_name
  ) AS subordinates 
FROM 
  employees e 
  INNER JOIN reporting_line rl ON e.manager_id = rl.employee_id;
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost port=15432 user=postgres database=dvdrental) at 0x20f614e5850>

In [5]:
sql = """
SELECT 
  subordinates 
FROM 
  reporting_line 
WHERE 
  employee_id = 10;
"""
run_sql(cursor, sql)

                                      subordinates
0  Michael North > Sarah Berry > Carolyn Henderson
